In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing important libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#reading the csv file
messages = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv")


In [ ]:
messages.head()

In [ ]:
messages.info()


In [ ]:
messages = messages.drop('Unnamed: 2', axis=1)
messages = messages.drop('Unnamed: 3', axis=1)
messages = messages.drop('Unnamed: 4', axis=1)

In [ ]:
messages.head()

In [ ]:
messages.describe()

In [ ]:
messages.groupby('v1').describe()

In [ ]:
messages['length'] = messages['v2'].apply(len)

In [ ]:
messages.head()

In [ ]:
sns.histplot(messages['length'], bins=100)

In [ ]:
messages.hist(column='length', by='v1', bins=60, figsize=(12,4))

In [ ]:
from sklearn.model_selection import train_test_split
msg_train, msg_test, label_train, label_test = train_test_split(messages['v2'], messages['v1'], test_size=0.3)
import string
from nltk.corpus import stopwords

In [ ]:
def text_process(mess):
    no_punc = [char for char in mess if char not in string.punctuation]
    no_punc = "".join(no_punc)
    return [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
messages.head()

In [ ]:
#applying function
msg_train.head(5).apply(text_process)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
pipeline.fit(msg_train,label_train)

In [ ]:
predictions = pipeline.predict(msg_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions,label_test))